In [1]:
print("hello world")

hello world


### Importing the Libarires

In [63]:
import random
import sys
import os
import time
import codecs
import collections
import numpy
import numpy as np
import pandas as pd
from textblob import TextBlob
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM
from tensorflow.keras.callbacks import ModelCheckpoint
from nltk.tokenize import sent_tokenize, word_tokenize
from keras.utils import to_categorical
# import scipy
# from scipy import spatial
from nltk.tokenize.toktok import ToktokTokenizer
import re
tokenizer = ToktokTokenizer()

In [36]:
df = pd.read_csv(
    "./Dataset./news_articles.csv", encoding='ISO-8859-1')

df.isnull()
df.isnull().sum().sum()
df.dropna(inplace=True)
print(df.info())

In [53]:

# df['text'].apply(lambda x: str(TextBlob(x).correct()))
Text_data = df['text'].values

list_data = Text_data.tolist()
print(list_data[:3])

<class 'pandas.core.frame.DataFrame'>
Index: 2045 entries, 0 to 2045
Data columns (total 12 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   author                   2045 non-null   object 
 1   published                2045 non-null   object 
 2   title                    2045 non-null   object 
 3   text                     2045 non-null   object 
 4   language                 2045 non-null   object 
 5   site_url                 2045 non-null   object 
 6   main_img_url             2045 non-null   object 
 7   type                     2045 non-null   object 
 8   label                    2045 non-null   object 
 9   title_without_stopwords  2045 non-null   object 
 10  text_without_stopwords   2045 non-null   object 
 11  hasImage                 2045 non-null   float64
dtypes: float64(1), object(11)
memory usage: 207.7+ KB
None
['print they should pay all the back all the money plus interest the entire family

In [54]:

TextData = ''.join(list_data)

TextData = TextData.replace('\n', '')
TextData = TextData.lower()

# Convert to lowercase and remove special characters
TextData = re.sub(r'(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])', ' ', TextData)
TextData = re.sub(r'[^a-zA-z0-9\s]', '', ''.join(TextData))

# Lemmatize words
lemmatizer = WordNetLemmatizer()
TextData = ' '.join([lemmatizer.lemmatize(word) for word in TextData.split()])

print(TextData[:100])

print they should pay all the back all the money plus interest the entire family and everyone who ca


In [56]:
# Tokenizing
tokens = tokenizer.tokenize(TextData)
tokens = [token.strip() for token in tokens]

# get the distinct words and sort it
word_counts = collections.Counter(tokens)
word_c = len(word_counts)
print('Total Word Cound : ', word_c)

distinct_words = [x[0] for x in word_counts.most_common()]
distinct_words_sorted = list(sorted(distinct_words))
# Generate indexing for all words
word_index = {x: i for i, x in enumerate(distinct_words_sorted)}
# decide on sentence length
sentence_length = 25

Total Word Cound :  45923


## prepare the dataset of input to output pairs encoded as integers
#### Generate the data for the model
input = the input sentence to the model with index

output = output of the model with index

In [72]:
InputData = []
OutputData = []
for i in range(0, word_c - sentence_length, 1):
    X = tokens[i:i + sentence_length]
    Y = tokens[i + sentence_length]
    InputData.append([word_index[char] for char in X])
    OutputData.append(word_index[Y])

print(InputData[:1])
print("\n")
print(OutputData[:1])

[[31587, 40587, 36766, 29734, 1168, 40416, 3273, 1168, 40416, 26115, 30691, 20617, 40416, 12966, 14226, 1630, 13543, 44714, 5854, 19855, 44986, 40441, 27062, 41018, 3719]]


[10260]


In [73]:
# Generate X
X = numpy.reshape(InputData, (len(InputData), sentence_length, 1))

# One hot encode the output variable
Y = to_categorical(OutputData)

print(f'length X : {len(X)} | length Y : {len(Y)}')

length X : 45898 | length Y : 45898


In [65]:
# define the LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(Y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

# define the checkpoint
file_name_path = "weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(
    file_name_path, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks = [checkpoint]

In [66]:
# fit the model
model.fit(X, Y, epochs=5, batch_size=128, callbacks=callbacks)

Epoch 1/5
359/359 [==============================] - ETA: 0s - loss: 7.8245
Epoch 1: loss improved from inf to 7.82449, saving model to weights-improvement-01-7.8245.hdf5


c:\Users\patil\OneDrive\Desktop\Python Projects\Text Generative AI\venv\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


359/359 [==============================] - 183s 492ms/step - loss: 7.8245
Epoch 2/5
359/359 [==============================] - ETA: 0s - loss: 7.2877
Epoch 2: loss improved from 7.82449 to 7.28767, saving model to weights-improvement-02-7.2877.hdf5
359/359 [==============================] - 153s 425ms/step - loss: 7.2877
Epoch 3/5
359/359 [==============================] - ETA: 0s - loss: 7.1463
Epoch 3: loss improved from 7.28767 to 7.14630, saving model to weights-improvement-03-7.1463.hdf5
359/359 [==============================] - 154s 429ms/step - loss: 7.1463
Epoch 4/5
359/359 [==============================] - ETA: 0s - loss: 7.0502
Epoch 4: loss improved from 7.14630 to 7.05020, saving model to weights-improvement-04-7.0502.hdf5
359/359 [==============================] - 148s 411ms/step - loss: 7.0502
Epoch 5/5
359/359 [==============================] - ETA: 0s - loss: 7.0113
Epoch 5: loss improved from 7.05020 to 7.01130, saving model to weights-improvement-05-7.0113.hdf5
359/

In [105]:
# load the network weights
file_name = "weights-improvement-05-7.0113.hdf5"
model.load_weights(file_name)
model.compile(loss='categorical_crossentropy', optimizer='adam')

## Predicting next word
We will randomly generate a sequence of words and input to the model 
and see what it predicts.

In [106]:
# Generating random sequence
# start = numpy.random.randint(0, len(InputData))
start = 103
input_sent = InputData[start]
# input_sent = InputData[start]

# Generate index of the next word of the email
X = numpy.reshape(input_sent, (1, len(input_sent), 1))

predict_word = model.predict(X, verbose=0)
index = numpy.argmax(predict_word)

print(input_sent)
print("\n")
print(index)

[20732, 6187, 29744, 41018, 20889, 28308, 8794, 36582, 27062, 41018, 30633, 40416, 40374, 36582, 12318, 5978, 33050, 33358, 41018, 1832, 28730, 21765, 30552, 9960, 40416]


40416


In [107]:
# Convert these indexes back to words

word_index_rev = dict((i, c) for i, c in enumerate(tokens))
result = word_index_rev[index]
sent_in = [word_index_rev[value] for value in input_sent]

print(sent_in)
print("\n")
print(result)

['on', 'to', 'so', 'and', 'partnership', 'that', 'in', 'credit', 'halfjokingly', 'and', 'they', 'even', 'been', 'credit', 'evidence', 'comment', 'getting', 'of', 'and', 'thing', 'richard', 'again', 'attendance', 'democracy', 'even']


even


In [ ]:

# Define hyperparameters
MAX_NB_WORDS = 20000
MAX_SEQUENCE_LENGTH = 100
EMBEDDING_DIM = 100
BATCH_SIZE = 32
EPOCHS = 10
vocab_size = len(tokenizer.word_index) + 1
Model_Results = list()

Creating Embedding Matrix

In [ ]:
# Load word embeddings
embeddings_index = {}
with open('/content/drive/MyDrive/Fresh_Project/Asserts/glove.6B.100d.txt') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs

# Create embedding matrix
num_words = min(MAX_NB_WORDS, len(word_index))
embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))
for word, i in word_index.items():
    if i >= MAX_NB_WORDS:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector